# Municipal Corporation Multi-Agent System

This notebook demonstrates a 3-agent pipeline for municipal project management:

1. **Formation Agent**: Analyzes citizen issues and creates project candidates
2. **Governance Agent**: Allocates budget and approves/rejects projects
3. **Scheduling Agent**: Creates execution schedules with resource constraints

## Setup

In [1]:
!pip install pandas numpy -q

import sqlite3
import json
import pandas as pd
from dataclasses import dataclass, field
from datetime import datetime



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# ---- Database path ----
DB_PATH = "city_planning.db"

# ---- City configuration ----
CITY_PROFILE = {
    "city_name": "Metroville",
    "population": 2_500_000,  # Changed to match original
    "quarterly_budget": 75_000_000  # Changed to match original default ($75M)
}

# ---- Risk thresholds ----
RISK_THRESHOLDS = {
    "high_population": 100_000,
    "high_complaints": 75,
    "high_risk_score": 3  # Only create projects for risk_score >= 3
}

# ---- Risk weights ----
RISK_WEIGHTS = {
    "safety_risk": 3,
    "legal_mandate": 3,
    "population_impact": 1,  # Changed to match original
    "complaint_volume": 1
}

# ---- Crew mapping ----
CREW_MAPPING = {
    "Water": "water_crew",
    "Health": "electrical_crew",
    "Disaster Management": "construction_crew",
    "Infrastructure": "construction_crew",
    "Recreation": "general_crew",
    "Education": "general_crew"
}


In [3]:
def init_database():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.executescript("""
    CREATE TABLE IF NOT EXISTS issues (
        issue_id INTEGER PRIMARY KEY,
        title TEXT,
        category TEXT,
        description TEXT,
        source TEXT,
        status TEXT
    );

    CREATE TABLE IF NOT EXISTS issue_signals (
        issue_id INTEGER,
        population_affected INTEGER,
        complaint_count INTEGER,
        safety_risk INTEGER,
        legal_mandate INTEGER,
        estimated_cost REAL,
        urgency_days INTEGER
    );

    CREATE TABLE IF NOT EXISTS project_candidates (
        project_id INTEGER PRIMARY KEY,
        issue_id INTEGER,
        title TEXT,
        scope TEXT,
        estimated_cost REAL,
        estimated_weeks INTEGER,
        required_crew_type TEXT,
        crew_size INTEGER,
        risk_score REAL,
        feasibility_score REAL,
        created_by TEXT
    );

    CREATE TABLE IF NOT EXISTS portfolio_decisions (
        decision_id INTEGER PRIMARY KEY,
        project_id INTEGER,
        decision TEXT,
        allocated_budget REAL,
        priority_rank INTEGER,
        rationale TEXT,
        decided_by TEXT
    );

    CREATE TABLE IF NOT EXISTS resource_calendar (
        resource_id INTEGER PRIMARY KEY AUTOINCREMENT,
        resource_type TEXT,
        week_number INTEGER,
        year INTEGER,
        capacity INTEGER,
        allocated INTEGER DEFAULT 0,
        UNIQUE(resource_type, week_number, year)
    );

    CREATE TABLE IF NOT EXISTS schedule_tasks (
        task_id INTEGER PRIMARY KEY,
        project_id INTEGER,
        start_week INTEGER,
        end_week INTEGER,
        resource_type TEXT,
        crew_assigned INTEGER,
        created_by TEXT
    );

    CREATE TABLE IF NOT EXISTS audit_log (
        event_id INTEGER PRIMARY KEY,
        event_type TEXT,
        agent_name TEXT,
        payload TEXT,
        timestamp TEXT
    );
    """)

    conn.commit()
    conn.close()

init_database()


In [4]:
def seed_data():
    """Seed data matching the original codebase for consistent outputs."""
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Clear existing data first (safe for reruns)
    cur.execute("DELETE FROM issues")
    cur.execute("DELETE FROM issue_signals")

    # Sample Issues - matching original seed_sample_data()
    issues_data = [
        (1, "Major Water Pipeline Rupture", "Water", 
         "Critical water main break affecting downtown area", "emergency_report", "OPEN"),
        (2, "Hospital Power Backup Failure", "Health",
         "Primary backup generator at City Hospital non-functional", "facility_inspection", "OPEN"),
        (3, "Urban Flooding in Low-Lying Areas", "Disaster Management",
         "Recurring flooding in Districts 4 and 7 during monsoon", "citizen_complaint", "OPEN"),
        (4, "Pothole Complaints in Residential Zones", "Infrastructure",
         "Multiple potholes reported on Main St and Oak Ave", "citizen_complaint", "OPEN"),
        (5, "Public Park Renovation Delay", "Recreation",
         "Central Park playground equipment outdated", "council_request", "OPEN"),
        (6, "Street Light Outages", "Infrastructure",
         "Multiple street lights non-functional in Sector 12", "citizen_complaint", "OPEN"),
        (7, "School Zone Safety Improvements", "Education",
         "Need for crosswalks and speed bumps near Lincoln Elementary", "citizen_complaint", "OPEN"),
    ]

    # Signals: (issue_id, population, complaints, safety, legal, est_cost, urgency_days)
    signals_data = [
        (1, 450000, 1200, 1, 1, 45000000, 7),    # Critical - affects half the city
        (2, 180000, 300, 1, 1, 12000000, 14),    # Critical - hospital safety
        (3, 600000, 900, 1, 0, 60000000, 30),    # High - monsoon preparedness
        (4, 80000, 40, 0, 0, 4000000, 60),       # Medium - quality of life
        (5, 15000, 12, 0, 0, 2500000, 180),      # Low - can wait
        (6, 25000, 85, 1, 0, 800000, 45),        # Medium-High - safety concern
        (7, 5000, 150, 1, 0, 500000, 30),        # Medium - child safety
    ]

    cur.executemany(
        "INSERT INTO issues (issue_id, title, category, description, source, status) VALUES (?, ?, ?, ?, ?, ?)",
        issues_data
    )

    cur.executemany(
        "INSERT INTO issue_signals (issue_id, population_affected, complaint_count, safety_risk, legal_mandate, estimated_cost, urgency_days) VALUES (?, ?, ?, ?, ?, ?, ?)",
        signals_data
    )

    conn.commit()
    conn.close()
    print("✓ Sample data seeded")


In [5]:
import sqlite3
import json
from dataclasses import dataclass, field
from typing import Optional
from datetime import datetime

@dataclass
class MunicipalContext:
    """
    Shared context object for all agents in the pipeline.
    """

    db_path: str = DB_PATH
    city_name: str = field(default_factory=lambda: CITY_PROFILE["city_name"])
    population: int = field(default_factory=lambda: CITY_PROFILE["population"])
    quarterly_budget: float = field(default_factory=lambda: CITY_PROFILE["quarterly_budget"])
    planning_horizon_weeks: int = 12
    


    risk_thresholds: dict = field(default_factory=lambda: RISK_THRESHOLDS.copy())
    risk_weights: dict = field(default_factory=lambda: RISK_WEIGHTS.copy())
    crew_mapping: dict = field(default_factory=lambda: CREW_MAPPING.copy())

    # ---------- DB helpers ----------
    def get_connection(self):
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        return conn

    def reset_resource_calendar(self):
      self.execute("UPDATE resource_calendar SET allocated = 0")


    def query(self, sql, params=()):
        conn = self.get_connection()
        try:
            cur = conn.execute(sql, params)
            rows = cur.fetchall()
            return [dict(row) for row in rows]
        finally:
            conn.close()

    def execute(self, sql, params=()):
        conn = self.get_connection()
        try:
            cur = conn.execute(sql, params)
            conn.commit()
            return cur.lastrowid
        finally:
            conn.close()

    def execute_many(self, sql, params_list):
        conn = self.get_connection()
        try:
            conn.executemany(sql, params_list)
            conn.commit()
        finally:
            conn.close()


    def project_candidate_exists(self, issue_id: int) -> bool:
      result = self.query(
          "SELECT 1 FROM project_candidates WHERE issue_id = ? LIMIT 1",
          (issue_id,)
      )
      return bool(result)


    # ---------- Audit ----------
    def log_audit(self, event_type, agent_name, payload):
        self.execute(
            "INSERT INTO audit_log (event_type, agent_name, payload, timestamp) VALUES (?, ?, ?, ?)",
            (event_type, agent_name, json.dumps(payload), datetime.now().isoformat())
        )

    # ---------- Issue queries ----------
    def get_open_issues(self):
        return self.query("""
            SELECT
                i.issue_id, i.title, i.category, i.description, i.source,
                s.population_affected, s.complaint_count, s.safety_risk,
                s.legal_mandate, s.estimated_cost, s.urgency_days
            FROM issues i
            LEFT JOIN issue_signals s ON i.issue_id = s.issue_id
            WHERE i.status = 'OPEN'
            ORDER BY s.urgency_days ASC
        """)

    def get_issue_by_id(self, issue_id: int):
        """Fetch a single issue by ID."""
        results = self.query("""
            SELECT 
                i.issue_id, i.title, i.category, i.description,
                s.population_affected, s.complaint_count, 
                s.safety_risk, s.legal_mandate, s.estimated_cost, s.urgency_days
            FROM issues i
            LEFT JOIN issue_signals s ON i.issue_id = s.issue_id
            WHERE i.issue_id = ?
        """, (issue_id,))
        return results[0] if results else None

    def get_portfolio_decisions(self):
      """
      Fetch all portfolio decisions with project details.
      """
      return self.query("""
          SELECT
              pd.decision_id,
              pd.project_id,
              pd.decision,
              pd.allocated_budget,
              pd.priority_rank,
              pd.rationale,
              pd.decided_by,
              pc.title
          FROM portfolio_decisions pd
          JOIN project_candidates pc
              ON pd.project_id = pc.project_id
          ORDER BY pd.priority_rank ASC
      """)


    def insert_portfolio_decision(self,project_id: int, decision: str, allocated_budget: float, priority_rank: int,rationale: str):
      """Insert a portfolio decision made by the governance agent."""
      return self.execute("""
          INSERT INTO portfolio_decisions
          (project_id, decision, allocated_budget, priority_rank, rationale, decided_by)
          VALUES (?, ?, ?, ?, ?, 'governance_agent')
      """, (
          project_id,
          decision,
          allocated_budget,
          priority_rank,
          rationale
    ))

    def get_system_summary(self):
      """
      Return a high-level summary of the system state.
      """
      candidates = self.get_project_candidates()
      decisions = self.get_portfolio_decisions()
      tasks = self.get_schedule_tasks()

      approved = [d for d in decisions if d.get("decision") == "APPROVED"]

      return {
          "city": self.city_name,
          "quarterly_budget": self.quarterly_budget,
          "planning_horizon_weeks": self.planning_horizon_weeks,
          "open_issues": len(self.get_open_issues()),
          "project_candidates": len(candidates),
          "approved_projects": len(approved),
          "scheduled_tasks": len(tasks),
          "total_allocated_budget": sum(
              d.get("allocated_budget", 0) for d in approved
          )
      }



    def get_schedule_tasks(self):
      """Fetch all scheduled tasks with project title."""
      return self.query("""
          SELECT st.*, pc.title
          FROM schedule_tasks st
          JOIN project_candidates pc ON st.project_id = pc.project_id
          ORDER BY st.start_week, st.project_id
      """)

    def insert_schedule_task(self, project_id, start_week, end_week, resource_type, crew_assigned):
        """Insert a scheduled task."""
        return self.execute("""
            INSERT INTO schedule_tasks 
            (project_id, start_week, end_week, resource_type, crew_assigned, created_by)
            VALUES (?, ?, ?, ?, ?, 'scheduling_agent')
        """, (project_id, start_week, end_week, resource_type, crew_assigned))


    # ---------- Project candidates ----------
    def get_project_candidates(self):
        return self.query("SELECT * FROM project_candidates ORDER BY risk_score DESC")

    def insert_project_candidate(
        self,
        issue_id,
        title,
        scope,
        estimated_cost,
        estimated_weeks,
        required_crew_type,
        crew_size,
        risk_score,
        feasibility_score=1.0
    ):
        return self.execute("""
            INSERT INTO project_candidates
            (issue_id, title, scope, estimated_cost, estimated_weeks,
             required_crew_type, crew_size, risk_score, feasibility_score, created_by)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, 'formation_agent')
        """, (
            issue_id, title, scope, estimated_cost, estimated_weeks,
            required_crew_type, crew_size, risk_score, feasibility_score
        ))

    def get_approved_projects(self):
      """
      Fetch only APPROVED projects with full details
      needed for scheduling.
      """
      return self.query("""
          SELECT
              pd.project_id,
              pd.allocated_budget,
              pd.priority_rank,
              pc.title,
              pc.estimated_weeks,
              pc.required_crew_type,
              pc.crew_size
          FROM portfolio_decisions pd
          JOIN project_candidates pc
              ON pd.project_id = pc.project_id
          WHERE pd.decision = 'APPROVED'
          ORDER BY pd.priority_rank ASC
      """)

    # ---------- Risk logic ----------
    def compute_risk_score(self, signal):
        score = 0.0

        if signal.get("safety_risk"):
            score += self.risk_weights["safety_risk"]

        if signal.get("legal_mandate"):
            score += self.risk_weights["legal_mandate"]

        if signal.get("population_affected", 0) >= self.risk_thresholds["high_population"]:
            score += self.risk_weights["population_impact"]

        if signal.get("complaint_count", 0) >= self.risk_thresholds["high_complaints"]:
            score += self.risk_weights["complaint_volume"]

        return score

    def get_available_capacity(self, resource_type: str, week: int) -> int:
      """
      Get available capacity for a resource type in a given week.
      Returns 0 if no calendar entry exists.
      """
      results = self.query("""
          SELECT capacity - allocated AS available
          FROM resource_calendar
          WHERE resource_type = ?
            AND week_number = ?
            AND year = 2025
      """, (resource_type, week))

      return results[0]["available"] if results else 0



    def allocate_resource(self, resource_type: str, week: int, units: int) -> bool:
      """
      Allocate resource units for a given week if capacity allows.
      Returns True if allocation succeeds.
      """
      available = self.get_available_capacity(resource_type, week)

      if available >= units:
          self.execute("""
              UPDATE resource_calendar
              SET allocated = allocated + ?
              WHERE resource_type = ?
                AND week_number = ?
                AND year = 2025
          """, (units, resource_type, week))
          return True

      return False

    def get_crew_type(self, category):
        return self.crew_mapping.get(category, "general_crew")


In [6]:
class FormationAgent:
    """
    Agent 1: Project Formation Agent

    Responsibilities:
    - Read OPEN issues and their signals
    - Compute risk scores
    - Create structured project candidates
    - Persist candidates to the database
    """

    name = "formation_agent"

    def __init__(self, ctx: MunicipalContext):
        self.ctx = ctx

    def run(self):
        issues = self.ctx.get_open_issues()
        created_projects = []

        for issue in issues:
            # --- Risk score computation ---
            risk_score = self.ctx.compute_risk_score(issue)

            # --- Only create projects for HIGH RISK issues (risk_score >= 3) ---
            if risk_score < self.ctx.risk_thresholds.get("high_risk_score", 3):
                continue

            # --- Estimate cost & duration (matching original heuristics) ---
            estimated_cost = issue.get("estimated_cost", 20000) or 0
            # Use cost-based heuristics like original
            if estimated_cost >= 50_000_000:
                estimated_weeks = 8
                crew_size = 3
            elif estimated_cost >= 10_000_000:
                estimated_weeks = 4
                crew_size = 2
            elif estimated_cost >= 1_000_000:
                estimated_weeks = 2
                crew_size = 2
            else:
                estimated_weeks = 1
                crew_size = 1

            # --- Crew requirements ---
            crew_type = self.ctx.get_crew_type(issue.get("category", "General"))

            # --- Project scope ---
            scope = (
                f"Resolve issue '{issue['title']}' "
                f"affecting {issue.get('population_affected', 'unknown')} people"
            )

            # --- Insert project candidate ---
            # 🔒 Deduplication guard
            if self.ctx.project_candidate_exists(issue["issue_id"]):
                # Skip creating duplicate project
                continue

            # --- Insert project candidate ---
            project_id = self.ctx.insert_project_candidate(
                issue_id=issue["issue_id"],
                title=issue["title"],
                scope=scope,
                estimated_cost=estimated_cost,
                estimated_weeks=estimated_weeks,
                required_crew_type=crew_type,
                crew_size=crew_size,
                risk_score=risk_score,
                feasibility_score=1.0
            )

            created_projects.append(project_id)

            # --- Audit log ---
            self.ctx.log_audit(
                event_type="PROJECT_CREATED",
                agent_name=self.name,
                payload={
                    "issue_id": issue["issue_id"],
                    "project_id": project_id,
                    "risk_score": risk_score
                }
            )

        return {
            "projects_created": len(created_projects),
            "project_ids": created_projects
        }

## Agent 2: Governance Agent

Allocates budget and approves/rejects projects based on risk scores and budget constraints.
    """
    Agent 3: Scheduling Agent

    Responsibilities:
    - Take APPROVED projects
    - Assign start/end weeks
    - Allocate crews based on availability
    - Persist schedule to database
    """

    name = "scheduling_agent"

    def __init__(self, ctx: MunicipalContext):
        self.ctx = ctx

    def run_greedy_scheduler_engine(self):
        projects = self.ctx.get_approved_projects()
        horizon = self.ctx.planning_horizon_weeks

        projects_sorted = sorted(projects, key=lambda x: x["priority_rank"])
        schedule = []
        infeasible = []

        for p in projects_sorted:
            scheduled = False
            for start_week in range(1, horizon - p["estimated_weeks"] + 2):
                if all(
                    self.ctx.get_available_capacity(p["required_crew_type"], w) >= p["crew_size"]
                    for w in range(start_week, start_week + p["estimated_weeks"])
                ):
                    for w in range(start_week, start_week + p["estimated_weeks"]):
                        self.ctx.allocate_resource(p["required_crew_type"], w, p["crew_size"])

                    schedule.append({
                        "project_id": p["project_id"],
                        "title": p["title"],
                        "start_week": start_week,
                        "end_week": start_week + p["estimated_weeks"] - 1,
                        "resource_type": p["required_crew_type"],
                        "crew_assigned": p["crew_size"],
                    })
                    scheduled = True
                    break

            if not scheduled:
                infeasible.append(p)

        return {
            "scheduled": schedule,
            "infeasible": infeasible
        }


In [8]:
def seed_resource_calendar():
    """Seed resource calendar matching original capacities."""
    conn = sqlite3.connect(DB_PATH, timeout=30)
    cur = conn.cursor()

    # Clear existing data first (safe re-run)
    cur.execute("DELETE FROM resource_calendar")

    # Resource types and capacities matching original
    resource_types = ["water_crew", "electrical_crew", "construction_crew", "general_crew"]
    capacities = {"water_crew": 3, "electrical_crew": 2, "construction_crew": 5, "general_crew": 4}
    
    rows = []
    for week in range(1, 13):
        for rtype in resource_types:
            rows.append((rtype, week, 2025, capacities[rtype], 0))

    # Fixed: Specify columns explicitly (resource_id auto-increments)
    cur.executemany(
        "INSERT INTO resource_calendar (resource_type, week_number, year, capacity, allocated) VALUES (?, ?, ?, ?, ?)",
        rows
    )

    conn.commit()
    conn.close()
    print("✓ Resource calendar seeded")


## Agent 3: Scheduling Agent

Creates execution schedules for approved projects with resource constraints.
    """Agent 3: Creates execution schedules for approved projects."""
    
    name = "scheduling_agent"
    
    def __init__(self, ctx: MunicipalContext):
self.ctx = ctx
    
    def run(self):
projects = self.ctx.get_approved_projects()
horizon = self.ctx.planning_horizon_weeks
projects_sorted = sorted(projects, key=lambda x: x["priority_rank"])
schedule = []
infeasible = []

for p in projects_sorted:
    scheduled = False
    for start_week in range(1, horizon - p["estimated_weeks"] + 2):
        if all(
            self.ctx.get_available_capacity(p["required_crew_type"], w) >= p["crew_size"]
            for w in range(start_week, start_week + p["estimated_weeks"])
        ):
            for w in range(start_week, start_week + p["estimated_weeks"]):
                self.ctx.allocate_resource(p["required_crew_type"], w, p["crew_size"])
            
            schedule.append({
                "project_id": p["project_id"],
                "title": p["title"],
                "start_week": start_week,
                "end_week": start_week + p["estimated_weeks"] - 1,
                "resource_type": p["required_crew_type"],
                "crew_assigned": p["crew_size"],
            })
            scheduled = True
            break
    
    if not scheduled:
        infeasible.append(p)

# Save schedule to database
for s in schedule:
    self.ctx.insert_schedule_task(
        project_id=s["project_id"],
        start_week=s["start_week"],
        end_week=s["end_week"],
        resource_type=s["resource_type"],
        crew_assigned=s["crew_assigned"]
    )
    self.ctx.log_audit(
        event_type="TASK_SCHEDULED",
        agent_name=self.name,
        payload=s
    )

return {"scheduled": schedule, "infeasible": infeasible}


In [ ]:
class SchedulingAgent: """Agent 3: Creates execution schedules for approved projects.""" name = "scheduling_agent" def __init__(self, ctx: MunicipalContext): self.ctx = ctx def run(self): projects = self.ctx.get_approved_projects() horizon = self.ctx.planning_horizon_weeks projects_sorted = sorted(projects, key=lambda x: x["priority_rank"]) schedule = [] infeasible = [] for p in projects_sorted: scheduled = False for start_week in range(1, horizon - p["estimated_weeks"] + 2): if all( self.ctx.get_available_capacity(p["required_crew_type"], w) >= p["crew_size"] for w in range(start_week, start_week + p["estimated_weeks"]) ): for w in range(start_week, start_week + p["estimated_weeks"]): self.ctx.allocate_resource(p["required_crew_type"], w, p["crew_size"]) schedule.append({ "project_id": p["project_id"], "title": p["title"], "start_week": start_week, "end_week": start_week + p["estimated_weeks"] - 1, "resource_type": p["required_crew_type"], "crew_assigned": p["crew_size"], }) scheduled = True break if not scheduled: infeasible.append(p) # Save schedule to database for s in schedule: self.ctx.insert_schedule_task( project_id=s["project_id"], start_week=s["start_week"], end_week=s["end_week"], resource_type=s["resource_type"], crew_assigned=s["crew_assigned"] ) self.ctx.log_audit( event_type="TASK_SCHEDULED", agent_name=self.name, payload=s ) return {"scheduled": schedule, "infeasible": infeasible}

In [ ]:
class PlanningPipeline:
    """Orchestrates the execution of all agents."""
    
    def __init__(self, ctx: MunicipalContext):
        self.ctx = ctx
        self.formation_agent = FormationAgent(ctx)
        self.governance_agent = GovernanceAgent(ctx)
        self.scheduling_agent = SchedulingAgent(ctx)
            
    def run(self):
        print("▶ Running Formation Agent")
        formation_result = self.formation_agent.run()
        print(f"   Created {formation_result['projects_created']} project candidates")

        print("▶ Running Governance Agent")
        governance_result = self.governance_agent.run()
        print(f"   Approved: {governance_result['approved_projects']}, Rejected: {governance_result['rejected_projects']}")

        print("▶ Running Scheduling Agent")
        self.ctx.reset_resource_calendar()  # Reset before scheduling
        scheduling_result = self.scheduling_agent.run()
        print(f"   Scheduled: {len(scheduling_result['scheduled'])}, Infeasible: {len(scheduling_result['infeasible'])}")

        print("✅ Pipeline completed")

        return {
            "formation": formation_result,
            "governance": governance_result,
            "scheduling": scheduling_result,
            "summary": self.ctx.get_system_summary()
        }


    def reset_run_state(ctx: MunicipalContext):
        """Clear previous agent outputs before running pipeline."""
        conn = sqlite3.connect(ctx.db_path)
        cur = conn.cursor()
        cur.execute("DELETE FROM project_candidates")
        cur.execute("DELETE FROM portfolio_decisions")
        cur.execute("DELETE FROM schedule_tasks")
        conn.commit()
        conn.close()


## Main Function: Generate Plan from Budget

**Input**: Quarterly budget (float)  
**Output**: Complete project plan with approved projects, schedules, and summary

#     for s in schedule:
#         ctx.context.insert_schedule_task(
#         )

#         ctx.context.log_audit(
#         )

#     return f"Saved {len(schedule)} scheduled tasks."


In [ ]:
def generate_plan(budget: float):
    """
    Generate complete municipal project plan from budget.
    
    Args:
        budget: Quarterly budget in USD
        
    Returns:
        dict with approved projects, rejected projects, and summary
    """
    # Ensure database is initialized and seeded
    seed_data()
    seed_resource_calendar()
    
    # Create context with specified budget
    ctx = MunicipalContext(quarterly_budget=budget)
    
    # Reset previous run state
    reset_run_state(ctx)
    
    # Run pipeline
    pipeline = PlanningPipeline(ctx)
    results = pipeline.run()
    
    # Format results for output
    decisions = ctx.get_portfolio_decisions()
    schedules = {t["project_id"]: t for t in ctx.get_schedule_tasks()}
    
    approved_projects = []
    rejected_projects = []
    
    for d in decisions:
        if d["decision"] == "APPROVED":
            sched = schedules.get(d["project_id"])
            approved_projects.append({
                "Project": d["title"],
                "Priority": d["priority_rank"],
                "Budget Allocated": f"${d['allocated_budget']:,.0f}",
                "Schedule": (
                    f"Week {sched['start_week']} - Week {sched['end_week']}"
                    if sched else "Not scheduled (resource constraints)"
                ),
                "Crew Type": sched.get("resource_type", "N/A") if sched else "N/A",
                "Rationale": d.get("rationale", "")
            })
        else:
            rejected_projects.append({
                "Project": d["title"],
                "Reason": d.get("rationale", "Rejected")
            })
    
    summary = ctx.get_system_summary()
    
    return {
        "Input Budget": f"${budget:,.0f}",
        "Summary": {
            "City": summary["city"],
            "Open Issues": summary["open_issues"],
            "Project Candidates Created": summary["project_candidates"],
            "Projects Approved": summary["approved_projects"],
            "Projects Rejected": len(rejected_projects),
            "Tasks Scheduled": summary["scheduled_tasks"],
            "Total Budget Allocated": f"${summary['total_allocated_budget']:,.0f}",
            "Budget Remaining": f"${budget - summary['total_allocated_budget']:,.0f}"
        },
        "Approved Projects": approved_projects,
        "Rejected Projects": rejected_projects
    }


✓ Sample data seeded
✓ Resource calendar seeded
▶ Running Formation Agent
   Created 5 project candidates
▶ Running Governance Agent
   Approved: 4, Rejected: 1
▶ Running Scheduling Agent
   Scheduled: 4, Infeasible: 0
✅ Pipeline completed
MUNICIPAL PROJECT PLAN

Input Budget: $75,000,000

Summary:
  City: Metroville
  Open Issues: 7
  Project Candidates Created: 5
  Projects Approved: 4
  Projects Rejected: 1
  Tasks Scheduled: 4
  Total Budget Allocated: $58,300,000
  Budget Remaining: $16,700,000

APPROVED PROJECTS
                           Project  Priority Budget Allocated  \
0     Major Water Pipeline Rupture         1      $45,000,000   
1    Hospital Power Backup Failure         2      $12,000,000   
2  School Zone Safety Improvements         3         $500,000   
3             Street Light Outages         4         $800,000   

          Schedule          Crew Type           Weather  \
0  Week 5 - Week 8         water_crew  ☀️ Clear weather   
1  Week 1 - Week 4    electrical

## Detailed Results View

View detailed project information as DataFrames.


In [13]:
# View detailed results as DataFrames
ctx = MunicipalContext()

print("Project Candidates:")
print(pd.DataFrame(ctx.get_project_candidates()))

print("\n\nPortfolio Decisions:")
print(pd.DataFrame(ctx.get_portfolio_decisions()))

print("\n\nSchedule Tasks:")
print(pd.DataFrame(ctx.get_schedule_tasks()))


Project Candidates:
   project_id  issue_id                              title  \
0           1         1       Major Water Pipeline Rupture   
1           2         2      Hospital Power Backup Failure   
2           3         3  Urban Flooding in Low-Lying Areas   
3           4         7    School Zone Safety Improvements   
4           5         6               Street Light Outages   

                                               scope  estimated_cost  \
0  Resolve issue 'Major Water Pipeline Rupture' a...      45000000.0   
1  Resolve issue 'Hospital Power Backup Failure' ...      12000000.0   
2  Resolve issue 'Urban Flooding in Low-Lying Are...      60000000.0   
3  Resolve issue 'School Zone Safety Improvements...        500000.0   
4  Resolve issue 'Street Light Outages' affecting...        800000.0   

   estimated_weeks required_crew_type  crew_size  risk_score  \
0                4         water_crew          2         8.0   
1                4    electrical_crew         

---

## How It Works

1. **Formation Agent** analyzes open issues and computes risk scores based on:
   - Safety risk (3 points)
   - Legal mandate (3 points)
   - Population impact (1 point if >100K affected)
   - Complaint volume (1 point if >75 complaints)
   
   Only issues with risk_score >= 3 become project candidates.

2. **Governance Agent** allocates budget using a greedy knapsack approach:
   - First approves all legal mandate projects
   - Then approves by risk-per-dollar (value density)
   - Rejects projects that don't fit remaining budget

3. **Scheduling Agent** assigns start weeks to approved projects:
   - Respects resource capacity constraints
   - Prioritizes by priority rank
   - Minimizes makespan

The system ensures:
- ✅ Legal mandates are always prioritized
- ✅ Budget constraints are respected
- ✅ Resource availability is checked
- ✅ Complete audit trail of all decisions
